## Setup

必要なライブラリをインポートして、初期化を行います。

Import the required libraries and initialize them:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ros_numpy
import rospy
import tf
from gazebo_ros import gazebo_interface
from sensor_msgs.msg import LaserScan, PointCloud2
from geometry_msgs.msg import Pose, Quaternion ,TransformStamped

import sys

from utils_notebooks import *
from utils_task1 import *
import cv2
import os


In [6]:
## THIS PATH MUST BE SET BY USER ()

#_path_xml = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models/MODEL_NAME/model-1_4.sdf"
#_path_model = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models"   Tenshi Version


_path_model = "/home/oscar/Codes/catkin_mio_ws/src/tmc_wrs_gazebo_world/models"     #Catkin mio version 
_path_xml = _path_model+"/MODEL_NAME/model-1_4.sdf"  #Catkin mio version


model_database_template = """<sdf version="1.4">
  <world name="default">
    <include>
      <uri>model://MODEL_NAME</uri>
    </include>
  </world>
</sdf>"""

objs=os.listdir(_path_model)
objs.sort()



rvizを起動します．ロボットモデル、カメラ映像、ポイントクラウドが表示されています。

In [3]:

rospy.init_node("recognition")


In [4]:
rgbd = RGBD()

In [5]:
listener = tf.TransformListener()
broadcaster= tf.TransformBroadcaster()
import moveit_commander
import moveit_msgs.msg

head = moveit_commander.MoveGroupCommander('head')
whole_body=moveit_commander.MoveGroupCommander('whole_body_weighted')
arm =  moveit_commander.MoveGroupCommander('arm')

In [7]:
def add_object(name, size, pose, orientation):
    p = PoseStamped()
    p.header.frame_id = "odom"       # "head_rgbd_sensor_link"
    
    p.pose.position.x = pose[0]
    p.pose.position.y = pose[1]
    p.pose.position.z = pose[2]

    p.pose.orientation.x = orientation[0] * np.pi
    p.pose.orientation.w = orientation[1] * np.pi

    scene.add_box(name, p, size)


def publish_scene():
    add_object("shelf", [1.5, 0.04, 0.4], [2.5, 4.7, 0.78], [0.5, 0.5])
    add_object("shelf1", [1.5, 0.04, 0.4], [2.5, 4.7, 0.49], [0.5, 0.5])
    add_object("shelf2", [1.5, 0.04, 0.4], [2.5, 4.7, 0.18], [0.5, 0.5])
    add_object("shelf_wall", [1, 1, 0.04], [2.5, 4.9, 0.5], [0.5, 0.5])
    add_object("shelf_wall1", [.04, 1, 0.4], [2.7, 4.9, 0.5], [0.5, 0.5])
    add_object("shelf_wall2", [.04, 1, 0.4], [1.8, 4.9, 0.5], [0.5, 0.5])
    
    add_object("table_big", [1.7, 0.13, 0.7], [0.95, 1.9, 0.34], [0.5, 0.5])
    add_object("table_small", [0.5, 0.01, 0.4], [0.1, 1.9, 0.61], [0.5, 0.5])
    add_object("table_tray", [0.65, 0.01, 0.7], [1.8, -0.65, 0.4], [0.5, 0.5])    
    return True

In [9]:
scene = moveit_commander.PlanningSceneInterface()
scene_pub = rospy.Publisher('planning_scene',
                                         moveit_msgs.msg.PlanningScene,
                                         queue_size=5)


In [8]:
publish_scene()


True

In [9]:
arm.set_named_target('neutral')
arm.go()
head.set_named_target('neutral')
head.go()

True

In [10]:
wb=whole_body.get_current_pose()

In [11]:
whole_body.set_workspace((-1,-1,5,5))

In [12]:
scene = moveit_commander.PlanningSceneInterface()


In [12]:
wb=whole_body.get_current_joint_values()
wb[0],wb[1],wb[2]= 1,1,0#-0.5*np.pi
whole_body.set_joint_value_target(wb)
whole_body.go()

False

In [33]:
whole_body.wb=whole_body.get_current_joint_values()
wb[0],wb[1],wb[2]= 2,3,0#-0.5*np.pi
whole_body.set_joint_value_target(wb)
whole_body.go()

True

In [29]:
wb=whole_body.get_current_pose()
wb.pose.position.x=.9
wb.pose.position.y=1.6
wb.pose.position.z=.75
wb.pose.orientation.x=1
wb.pose.orientation.y=0
wb.pose.orientation.z=0
wb.pose.orientation.w=0

In [30]:
whole_body.set_pose_target(wb)

In [31]:
whole_body.go()

True

In [40]:
wb=whole_body.get_current_pose()
wb


header: 
  seq: 0
  stamp: 
    secs: 865
    nsecs: 614000000
  frame_id: "odom"
pose: 
  position: 
    x: 0.668927609595
    y: 1.48006914574
    z: 0.748947184566
  orientation: 
    x: 0.999998225969
    y: -0.00183592730117
    z: 6.67677733657e-05
    w: 0.000415899773274

In [41]:

wb.pose.position.y=1.6


In [ ]:
whole_body.go(wb)